# Calculating integrals over GTOs

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=5, linewidth=120)

In this example, we'll go over the high- and low-level machinery that GQCP offers in order to calculate integrals over Cartesian GTOs. We assume that you're familiar with the mathematical concepts of a _scalar basis_, a _shell_ and a _primitive_.

We'll start off by setting up a small molecular system.

In [2]:
molecule = gqcpy.Molecule([
    gqcpy.Nucleus(1, 0.0, 0.0, 0.0),
    gqcpy.Nucleus(1, 0.0, 0.0, 1.0)
])

## Primitive engines

Before tackling the first type of integrals (the overlap integrals), we'll have to zoom in a bit on how integrals are calculated. Let's dive in:
- Spin-orbitals are expanded in an underlying scalar basis.
- In order to compactify scalar bases, we use shells that group primitives according to their angular momentum.
- Therefore, in every shell, a _set_ of basis functions (of the same angular momentum) is implicitly defined.
- Every basis function is defined as a contraction (i.e. a linear combination, where the coefficients are called _contraction coefficients_) of primitives.
- Here, the primitives are Cartesian GTOs.

In GQCP, we define an _engine_ to be a computational object that is able to calculate integrals over _shells_, while a _primitive engine_ is defined to be a computational entity that can calculate integrals over _primitives_. 

In this example, we're taking the expansion of contracted GTOs in terms of their primitives for granted (using the implementations provided by the combination of `FunctionalOneElectronIntegralEngine` and `IntegralCalculator`), and we'll be focusing on calculating integrals over _primitives_.

Since we really don't want to manually read in a basis set, we'll use `RSpinOrbitalBasis`'s functionality to provide us with a scalar basis.

In [3]:
spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
scalar_basis = spin_orbital_basis.scalarBasis()
shell_set = scalar_basis.shellSet()  # A shell set is just a collection of shells.

## Overlap integrals

Let's get straight into it. Using the McMurchie-Davidson integral scheme, we can calculate the overlap integral over two primitives as follows.

In [4]:
def overlap_function(left, right):
    
    primitive_integral = 1.0
    
    # The overlap integral is separable in its three Cartesian components.
    for direction in [gqcpy.CartesianDirection.x, gqcpy.CartesianDirection.y, gqcpy.CartesianDirection.z]:
        i = left.cartesianExponents().value(direction)
        j = right.cartesianExponents().value(direction)
        
        a = left.gaussianExponent()
        b = right.gaussianExponent()
        p = a + b

        K = left.center()[direction]
        L = right.center()[direction]


        E = gqcpy.McMurchieDavidsonCoefficient(K, a, L, b)
        
        primitive_integral_1D = np.power(np.pi / p, 0.5) * E(i, j, 0)
        primitive_integral *= primitive_integral_1D


    return primitive_integral

We'll wrap this function in to a `FunctionalPrimitiveIntegralEngine`, and then supply it as the primitive engine that should be used in a `FunctionalOneElectronIntegralEngine`. We're doing this in order to use GQCP's internal handling of the shells and contractions through the `IntegralCalculator.calculate` call.

In [5]:
primitive_overlap_engine = gqcpy.FunctionalPrimitiveIntegralEngine_d(overlap_function)
overlap_engine = gqcpy.FunctionalOneElectronIntegralEngine_d(primitive_overlap_engine)

In [6]:
S = gqcpy.IntegralCalculator.calculate(overlap_engine, shell_set, shell_set)
print(S)

[[1.      0.79659]
 [0.79659 1.     ]]


We can verify our results by letting the spin-orbital basis quantize the overlap operator.

In [7]:
S_ref = spin_orbital_basis.quantizeOverlapOperator().parameters()
print(S_ref)

[[1.      0.79659]
 [0.79659 1.     ]]
